In [1]:
import extract
import pandas as pd
import os
import geopandas as gpd

In [2]:
import importlib
importlib.reload(extract)

<module 'extract' from 'd:\\Users\\User\\Downloads\\RA Work\\acs\\extract.py'>

In [3]:
# open state_fips
state_fips = pd.read_csv('state_fips.csv')

In [4]:
# for every folder in raw_tract_data
for folder in os.listdir('raw_tract_data'):
    # split folder into list by underscore
    folder_tokens = folder.split('_')
    
    # get state column from state_fips where code is third item in list as int
    state = state_fips[state_fips['code'] == int(folder_tokens[2])]['state'].values[0].lower()

    # if last item in folder is 'tract' year is 2000, otherwise it is 20 + last two digits
    year = '2020' if folder_tokens[-1] == 'tract' else '20' + folder_tokens[-1][-2:]

    # if tract file already exists, skip
    if os.path.exists(f'tract_polygons/{state}_tracts_{year}.csv'):
        continue

    # get the filepath of the shp file in the folder with the same name
    shp_file = os.path.join('raw_tract_data', folder, folder + '.shp')

    tract_geos = extract.read_tracts(shp_file)
    
    # save as acs\tract_polygons\berkeley_tracts_2010.csv
    tract_geos.to_csv(f'tract_polygons/{state}_tracts_{year}.csv', index=False)

In [46]:
# for alaska upper (weird KML)
gpd.io.file.fiona.drvsupport.supported_drivers['KML'] = 'rw'
districts = gpd.read_file('raw_geo_data\\alaska_upper_2022.kml', driver='KML')
# rename Name to district
districts = districts.rename(columns={'Name': 'district'})
# only keep district and geometry
districts = districts[['district', 'geometry']]
# only keep the part of the district field in the form 0200A
districts['district'] = districts['district'].str.extract(r'(\d{4}[A-Z])')
# only keep the last letter
districts['district'] = districts['district'].str[-1]
# save to geo_polygons folder as alaska_upper_districts_2022.csv
# districts.to_csv('geo_polygons/alaska_upper_districts_2022.csv', index=False)

In [26]:
# for SHP
# read in shp
districts = gpd.read_file(r'raw_geo_data\SLP_Wards_Precincts_ExportFeatures (1)\SLP_Wards_Precincts_ExportFeatures.shp')
districts

,PRECINCTID,WARD,NAME,Shape_Leng,Shape_Area,geometry
0,1,Ward 1,Ward 1 Precinct 1,20333.756490,2.562813e+07,"POLYGON ((-93.329 44.948, -93.330 44.948, -93...."
1,2,Ward 1,Ward 1 Precinct 2,20754.928395,2.202543e+07,"POLYGON ((-93.371 44.957, -93.371 44.957, -93...."
2,3,Ward 1,Ward 1 Precinct 3,24127.545702,1.802082e+07,"POLYGON ((-93.348 44.948, -93.348 44.948, -93...."
3,4,Ward 2,Ward 2 Precinct 4,20992.709171,2.450834e+07,"POLYGON ((-93.339 44.931, -93.341 44.931, -93...."
4,5,Ward 2,Ward 2 Precinct 5,24592.015759,2.603857e+07,"POLYGON ((-93.339 44.931, -93.339 44.929, -93...."
5,6,Ward 2,Ward 2 Precinct 6,29151.961619,3.228149e+07,"POLYGON ((-93.348 44.941, -93.348 44.941, -93...."
6,7,Ward 3,Ward 3 Precinct 7,18894.319657,2.080092e+07,"POLYGON ((-93.371 44.950, -93.370 44.950, -93...."
7,8,Ward 3,Ward 3 Precinct 8,26343.026097,1.867920e+07,"POLYGON ((-93.371 44.957, -93.371 44.957, -93...."
8,9,Ward 3,Ward 3 Precinct 9,25280.917334,2.238571e+07,"POLYGON ((-93.378 44.944, -93.378 44.942, -93...."
9,10,Ward 4,Ward 4 Precinct 10,30388.886418,2.688261e+07,"POLYGON ((-93.329 44.965, -93.329 44.965, -93...."


In [27]:
# only keep district and geometry
district_name = 'WARD'
year=2021
desc='st_louis_park_wards'
districts = districts[[district_name, 'geometry']]
# rename to district and convert to int
districts = districts.rename(columns={district_name: 'district'})
# set district to only last char
districts['district'] = districts['district'].str[-1]
districts['district'] = districts['district'].astype(int)
# combine polygons of same district
districts = districts.dissolve(by='district', as_index=False)

In [28]:
districts

,district,geometry
0,1,"POLYGON ((-93.329 44.948, -93.329 44.948, -93...."
1,2,"POLYGON ((-93.339 44.929, -93.339 44.928, -93...."
2,3,"POLYGON ((-93.377 44.937, -93.378 44.937, -93...."
3,4,"POLYGON ((-93.361 44.959, -93.362 44.959, -93...."


In [29]:
# save to geo_polygons folder
districts.to_csv(f'geo_polygons/{desc}_{year}.csv', index=False)